In [ ]:
!pip install swig
!pip install gym[box2d]

In [73]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_probability as tfp

In [74]:
import gym
from tqdm import tqdm_notebook
import numpy as np
from collections import deque

In [75]:
class PolicyNetwork(tf.keras.Model):
    def __init__(self, observation_space, action_space):
        super(PolicyNetwork, self).__init__()
        # Initialize layers
        self.input_layer = tf.keras.layers.Dense(128, input_dim=observation_space, activation='relu')
        self.output_layer = tf.keras.layers.Dense(action_space, activation='softmax')

    def call(self, inputs):
        # Forward pass
        x = self.input_layer(inputs)
        action_probs = self.output_layer(x)
        return action_probs

In [76]:
class StateValueNetwork(tf.keras.Model):
    def __init__(self, observation_space):
        super(StateValueNetwork, self).__init__()
        self.input_layer = layers.Dense(128, activation='relu', input_shape=(observation_space,))
        self.output_layer = layers.Dense(1, activation=None)  # typically no activation for value output

    def call(self, inputs):
        x = self.input_layer(inputs)
        state_value = self.output_layer(x)
        return state_value

In [77]:
def select_action(network, state):
    ''' Selects an action given current state in TensorFlow
    Args:
    - network (tf.keras.Model): network to process state
    - state (Array): Array of action space in an environment

    Return:
    - (int): action that is selected
    - (float): log probability of selecting that action given state and network
    '''
    # convert state to float tensor, add 1 dimension
    state = tf.convert_to_tensor(state, dtype=tf.float32)
    state = tf.expand_dims(state, 0)

    # use network to predict action probabilities
    action_probs = network(state)

    # sample an action using the probability distribution
    action_dist = tfp.distributions.Categorical(probs=action_probs)
    action = action_dist.sample()

    # compute log probability of the selected action
    log_prob = action_dist.log_prob(action)

    # return action and log probability
    return action.numpy()[0], log_prob.numpy()[0]

In [78]:
env = gym.make(
    "LunarLander-v2"
)

# Instantiate the networks
policy_network = PolicyNetwork(env.observation_space.shape[0], env.action_space.n)
stateval_network = StateValueNetwork(env.observation_space.shape[0])

# Initialize the optimizers
policy_optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
stateval_optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [79]:
#discount factor for future utilities
DISCOUNT_FACTOR = 0.99

#number of episodes to run
NUM_EPISODES = 500

#max steps per episode
MAX_STEPS = 10000

#score agent needs for environment to be solved
SOLVED_SCORE = 195

In [ ]:
policy_network.trainable_variables

[<tf.Variable 'policy_network_5/dense_18/kernel:0' shape=(8, 128) dtype=float32, numpy=
 array([[-0.04808192, -0.1171983 ,  0.06812976, ..., -0.15145896,
          0.00117312,  0.03061894],
        [ 0.17905344,  0.07582764, -0.15012689, ...,  0.08952861,
          0.15179394, -0.00740643],
        [ 0.08402084,  0.00120749,  0.08395649, ..., -0.05509317,
          0.08067052, -0.17857836],
        ...,
        [-0.03884403, -0.10525813,  0.14588685, ..., -0.19521789,
         -0.06740102,  0.192192  ],
        [-0.0200502 ,  0.1033379 ,  0.11397909, ...,  0.13289063,
         -0.14636038,  0.10663916],
        [ 0.05271553,  0.19778956,  0.06622379, ...,  0.11798503,
          0.11922942,  0.03053285]], dtype=float32)>,
 <tf.Variable 'policy_network_5/dense_18/bias:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0

In [80]:
import numpy as np
from collections import deque
from tqdm import tqdm

scores = []
recent_scores = deque(maxlen=100)

for episode in tqdm(range(NUM_EPISODES)):
    state = env.reset()
    done = False
    score = 0
    I = 1.0

    for step in range(MAX_STEPS):
        action, lp = select_action(policy_network, state)
        new_state, reward, done, _ = env.step(action)
        score += reward


        with tf.GradientTape() as policy_tape, tf.GradientTape() as value_tape:
            state_val = stateval_network(tf.convert_to_tensor([state], dtype=tf.float32))
            new_state_val = stateval_network(tf.convert_to_tensor([new_state], dtype=tf.float32))

            if done:
                new_state_val = tf.constant([[0.0]])

            # 计算损失
            val_loss = tf.reduce_mean(tf.square(reward + DISCOUNT_FACTOR * new_state_val - state_val))
            advantage = reward + DISCOUNT_FACTOR * tf.squeeze(new_state_val) - tf.squeeze(state_val)
            #advantage = tf.convert_to_tensor(advantage)
            lp_tensor = tf.convert_to_tensor(lp, dtype=tf.float32)
            policy_loss = -lp_tensor * advantage
            policy_loss *= I

        # 反向传播和优化
        gradients = policy_tape.gradient(policy_loss, policy_network.trainable_variables)
        policy_optimizer.apply_gradients(zip(gradients, policy_network.trainable_variables))

        gradients = value_tape.gradient(val_loss, stateval_network.trainable_variables)
        stateval_optimizer.apply_gradients(zip(gradients, stateval_network.trainable_variables))

        if done:
            break

        state = new_state
        I *= DISCOUNT_FACTOR

    scores.append(score)
    recent_scores.append(score)

    if np.mean(recent_scores) >= SOLVED_SCORE:
        break


  0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
  0%|          | 0/500 [00:00<?, ?it/s]


ValueError: No gradients provided for any variable: (['policy_network_6/dense_22/kernel:0', 'policy_network_6/dense_22/bias:0', 'policy_network_6/dense_23/kernel:0', 'policy_network_6/dense_23/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'policy_network_6/dense_22/kernel:0' shape=(8, 128) dtype=float32, numpy=
array([[ 0.19794314, -0.0693457 , -0.19295913, ...,  0.15815432,
        -0.05465674,  0.07426123],
       [ 0.1701522 , -0.03206649,  0.04759721, ...,  0.04679726,
        -0.10077876, -0.10581645],
       [ 0.17518122, -0.0837068 ,  0.19845034, ...,  0.06641267,
        -0.1842982 , -0.17843804],
       ...,
       [-0.11206261, -0.11143719, -0.04583035, ..., -0.12017128,
        -0.01050779,  0.14847769],
       [ 0.15997095, -0.02252905, -0.03360023, ...,  0.16476949,
        -0.10409091,  0.16908588],
       [-0.00198098,  0.02651316, -0.13287775, ..., -0.07062525,
         0.10262774, -0.20046265]], dtype=float32)>), (None, <tf.Variable 'policy_network_6/de

In [ ]:
policy_network.trainable_variables

[<tf.Variable 'policy_network_5/dense_18/kernel:0' shape=(8, 128) dtype=float32, numpy=
 array([[-0.04808192, -0.1171983 ,  0.06812976, ..., -0.15145896,
          0.00117312,  0.03061894],
        [ 0.17905344,  0.07582764, -0.15012689, ...,  0.08952861,
          0.15179394, -0.00740643],
        [ 0.08402084,  0.00120749,  0.08395649, ..., -0.05509317,
          0.08067052, -0.17857836],
        ...,
        [-0.03884403, -0.10525813,  0.14588685, ..., -0.19521789,
         -0.06740102,  0.192192  ],
        [-0.0200502 ,  0.1033379 ,  0.11397909, ...,  0.13289063,
         -0.14636038,  0.10663916],
        [ 0.05271553,  0.19778956,  0.06622379, ...,  0.11798503,
          0.11922942,  0.03053285]], dtype=float32)>,
 <tf.Variable 'policy_network_5/dense_18/bias:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0